In [ ]:
import pandas as pd
import yaml
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

from IPython.display import Markdown, display
import visualizer_helpers as vh

In [ ]:
# NOTE: SPECIALLY TAGGED PARAMETERS CELL
# Assignments in this cell may be overwritten at compile-time
# Otherwise, the below defaults to not filtering any zones
zone_set: str = 'all'
how_method: str = 'any'
affected_tazs: list = []
affected_mazs: list = []

In [ ]:
# get corresponding filter from zone_set parameter
single_filter_tazs, multi_filter_tazs = vh.get_filters(zone_set, how_method, affected_tazs)
single_filter_mazs, multi_filter_mazs = vh.get_filters(zone_set, how_method, affected_mazs)

In [ ]:
# file IO locations
with open('_quarto.yml') as f:
    config = yaml.safe_load(f)
base_dir = config['sources']['base']
build_dir = config['sources']['build']

In [ ]:
base_tour = pd.read_csv(f"{base_dir}/final_tours.csv", 
                        index_col='tour_id', 
                        usecols=['tour_id','tour_category','origin','destination','primary_purpose'])

base_tour_idx = base_tour.index

build_tour = pd.read_csv(f"{build_dir}/final_tours.csv", 
                         index_col='tour_id', 
                         usecols=['tour_id','tour_category','origin','destination','primary_purpose'])


In [ ]:
print(f"Filtering tours by origin and destination TAZ. Original tours in base: {len(base_tour)}\tbuild: {len(build_tour)}")
base_tour = base_tour[multi_filter_tazs([base_tour.origin, base_tour.destination])] # base tour in the filtered set

build_tour = build_tour[

    (build_tour.index.isin(base_tour_idx) # originally existed in base
    & build_tour.index.isin(base_tour.index))    # and was in the current set for the base

    |     # OR

    ((~build_tour.index.isin(base_tour_idx)) # is a new tour 
     & multi_filter_tazs([build_tour.origin, build_tour.destination])) # and it's in this set

     ]
print(f"After filtering, tours in base: {len(base_tour)}\tbuild: {len(build_tour)}")

In [ ]:
name_dict = {'escort': "Escorting", 
             'work': "Work", 
             'school': "School", 
             'othmaint': "Other-Maintenance",
             'othdiscr': "Other-Discretionary", 
             'shopping': "Shopping",
             'atwork': "At-work Subtour", 
             'eatout': "Eat Out", 
             'social' : "Social", 
             'univ': "University"}

## Non-Mandatory Tour Frequency

In [ ]:
print("Lorem ipsum") # Placeholder for WSP

## Non-Mandatory Tour Destinations

In [ ]:
df = base_tour[base_tour.tour_category == 'non_mandatory'].merge(build_tour[build_tour.tour_category == 'non_mandatory'],
                                                                 how='outer',
                                                                 left_index=True,
                                                                 right_index=True,
                                                                 suffixes=('_base','_build'))



In [ ]:
df.loc[df.destination_base == df.destination_build,'Case'] = "Unchanged"
df.loc[df.destination_base != df.destination_build,'Case'] = "Changed"

df.loc[df.destination_base.isna(),'Case'] = "Newly Created"
df.loc[df.destination_build.isna(),"Case"] = "Removed"


In [ ]:
df = df.Case.value_counts()

df

In [ ]:
fig = vh.create_pie_chart(df.to_frame(),["count"])
fig.show()

In [ ]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly

for key, value in name_dict.items():

    if len(base_tour[
        (base_tour.tour_category == 'non_mandatory')
        & (base_tour.primary_purpose == key)
        ]) == 0:
        continue
    
    print(f"#### {value}")
    df = base_tour[
        (base_tour.tour_category == 'non_mandatory')
        & (base_tour.primary_purpose == key)
        ].merge(build_tour[
                    (build_tour.tour_category == 'non_mandatory')
                    & (build_tour.primary_purpose == key)
                ],
                how='left',
                left_index=True,
                right_index=True,
                suffixes=('_base','_build'))
    
    df.loc[df.destination_base == df.destination_build,'Case'] = "Unchanged"
    df.loc[df.destination_base != df.destination_build,'Case'] = "Changed"

    df.loc[df.destination_base.isna(),'Case'] = "Newly Created"
    df.loc[df.destination_build.isna(),"Case"] = "Removed"

    
    df = df.Case.value_counts()

    print(str(df).replace("\n","<br>"))
    
    fig = vh.create_pie_chart(df.to_frame(),["count"])
    
    fig.show()
    display(Markdown(" "))

## Non-Mandatory Tour Scheduling

In [ ]:
print("Lorem ipsum") # Placeholder for WSP

## Tour Mode Choice

In [ ]:
print("Lorem ipsum") # Placeholder for CS

## At-Work Subtour Frequency

In [ ]:
print("Lorem ipsum") # Placeholder for WSP

## At-Work Subtour Destinations

In [ ]:
base_tour = pd.read_csv(f"{base_dir}/final_tours.csv", 
                        index_col='tour_id', 
                        usecols=['tour_id','tour_category','destination','primary_purpose'])

build_tour = pd.read_csv(f"{build_dir}/final_tours.csv", 
                         index_col='tour_id', 
                         usecols=['tour_id','tour_category','destination','primary_purpose'])


In [ ]:
df = base_tour[base_tour.tour_category == 'atwork'].merge(build_tour[build_tour.tour_category == 'atwork'],
                                                          how='outer',
                                                          left_index=True,
                                                          right_index=True,
                                                          suffixes=('_base','_build'))


In [ ]:
df.loc[df.destination_base == df.destination_build,'Case'] = "Unchanged"
df.loc[df.destination_base != df.destination_build,'Case'] = "Changed"

df.loc[df.destination_base.isna(),'Case'] = "Newly Created"
df.loc[df.destination_build.isna(),"Case"] = "Removed"


In [ ]:
df = df.Case.value_counts()

df

In [ ]:
fig = vh.create_pie_chart(df.to_frame(),["count"])
fig.show()

## At-Work Subtour Scheduling

In [ ]:
print("Lorem ipsum") # Placeholder for WSP

## At-Work Subtour Mode Choice

In [ ]:
print("Lorem ipsum") # Placeholder for CS

## Stop Frequency

In [ ]:
print("Lorem ipsum") # Placeholder for CS